# Wildfire Data Acquisition and Proximity Computation

This notebook analyzes wildfire impacts on Mesa, Arizona (population: 511,648 as of 2021) to inform policymakers about potential mitigation strategies for future wildfire effects. Mesa is located at 33.40°N 111.72°W in the state of Arizona. 

In this file, we focus on acquiring the data from USGS ["Combined Wildland Fire Datasets for the United States and Certain Territories, 1800s-Present"](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) dataset, calculating the proximity of the fire from Mesa, AZ, and the filtering the data on the conditions below. We also calculate the proximity of the fire from my assigned city of Mesa, AZ.

1. We considers only the last 60 years of wildland fire data (1964-2024)

2. We only include fires that are within 650 miles of Mesa, AZ


### License
A lot of the code/ markdown text below was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - August 16, 2024.

I have made slight modifications to the code as required for my project

### Preliminaries
First we start with some imports and some constant definitions.


This notebook has dependencies on [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html). Pyproj can be installed via pip. 
You will need to install this with pip/pip3 if you do not already have it.
After installing pip, you can run: !pip install {package name} to install any required packages

In [ ]:
#
#    IMPORTS
# 

#    Import some standard python modules
import json
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#

#    
import pandas as pd
#

**Step 1:** Load the wildfire data


Note: The USGS_Wildland_Fire_Combined_Dataset.json file is over 2 GB in size and couldn't be uploaded on GIT. In order to get that, you need to go the USGS ["Combined Wildland Fire Datasets for the United States and Certain Territories, 1800s-Present"](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) dataset and then download the zip folder, and extract the file.

In [ ]:
data = r".\Raw Data\USGS_Wildland_Fire_Combined_Dataset.json"

# Load the JSON file
with open(data, 'r') as file:
    gj_data = json.load(file)

In [ ]:
# Some part taken from professors file
gj_keys = list(gj_data.keys())
print("The loaded JSON dictionary has the following keys:")
print(gj_keys)
#
#    For all GeoJSON type things, the most important part of the file are the 'features'. 
#    In the case of the wildfire dataset, each feature is a polygon (ring) of points that define the bounary of a fire
#
count = 0
features = gj_data.get("features", [])
attributes = []
for curr_feature in features:
    attributes.append(curr_feature.get('attributes', {}))
    count += 1

print(f"Found {count} features in the variable 'gj_data' ")


df = pd.DataFrame(attributes)

# Display basic information about the DataFrame
print("DataFrame Info:")
print(df.info())

print("First few rows of data:")
print(df.head())


# Determine the range of years in the dataset
earliest_year = df['Fire_Year'].min()
latest_year = df['Fire_Year'].max()
print(f"Earliest year: {earliest_year}, Latest year: {latest_year}")

cutoff_year = 2024 - 60 # Most recent 60 years
filtered_wildfires = [fire for fire in gj_data['features'] if fire['attributes']["Fire_Year"] >= cutoff_year]
print(f"The total number of wildfires between {cutoff_year} and 2024: {len(filtered_wildfires)}")

The loaded JSON dictionary has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']
Found 135061 features in the variable 'gj_data' 
DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135061 entries, 0 to 135060
Data columns (total 30 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   OBJECTID                      135061 non-null  int64  
 1   USGS_Assigned_ID              135061 non-null  int64  
 2   Assigned_Fire_Type            135061 non-null  object 
 3   Fire_Year                     135061 non-null  int64  
 4   Fire_Polygon_Tier             135061 non-null  int64  
 5   Fire_Attribute_Tiers          135061 non-null  object 
 6   GIS_Acres                     135061 non-null  float64
 7   GIS_Hectares                  135061 non-null  float64
 8   Source_Datasets               135061 non-null  object 
 9   Listed_Fire_T

Now, we try to understand what is in the dataset

In [ ]:
# Taken from the professors code
#
#    Get the first item in the list of features
#
SLOT = 0
gj_feature = gj_data['features'][SLOT]
#
#    Print everyting in this dictionary (i.e., gj_feature) - it's long
#
print(f"The wildfire feature from slot '{SLOT}' of the loaded gj_data['features']")
print(json.dumps(gj_feature, indent=4))


# Output the keys and feature count to confirm data loading.
print("Loaded GeoJSON data keys:", list(gj_data.keys()))
print(f"Total features in GeoJSON data: {len(features)}")
print()

The wildfire feature from slot '0' of the loaded gj_data['features']
{
    "attributes": {
        "OBJECTID": 1,
        "USGS_Assigned_ID": 1,
        "Assigned_Fire_Type": "Wildfire",
        "Fire_Year": 1860,
        "Fire_Polygon_Tier": 1,
        "Fire_Attribute_Tiers": "1 (1)",
        "GIS_Acres": 3940.20708940724,
        "GIS_Hectares": 1594.5452365353703,
        "Source_Datasets": "Comb_National_NIFC_Interagency_Fire_Perimeter_History (1)",
        "Listed_Fire_Types": "Wildfire (1)",
        "Listed_Fire_Names": "Big Quilcene River (1)",
        "Listed_Fire_Codes": "No code provided (1)",
        "Listed_Fire_IDs": "",
        "Listed_Fire_IRWIN_IDs": "",
        "Listed_Fire_Dates": "Listed Other Fire Date(s): 2006-11-02 - NIFC DATE_CUR field (1)",
        "Listed_Fire_Causes": "",
        "Listed_Fire_Cause_Class": "Undetermined (1)",
        "Listed_Rx_Reported_Acres": null,
        "Listed_Map_Digitize_Methods": "Other (1)",
        "Listed_Notes": "",
        "Proce

In [ ]:
# Taken from the professors code
#  
#    Every feature has a 'geometry' which specifies geo coordinates that make up each geographic thing
#    In the case of the wildfire data, most wildfires are bounded shapes, circles, squares, etc. This is
#    represented by shapes called 'rings' in GeoJSON.
# 
# Get the geometry for the feature we pulled from the feature_list
gj_geometry = gj_feature['geometry']
# The largest shape (ring) is supposed to be item zero in the list of 'rings'
gj_bigest_ring = gj_geometry['rings'][0]

print(f"The largest ring of gj_feature['features'][{SLOT}]['rings'] consists of {len(gj_bigest_ring)} points.")

The largest ring of gj_feature['features'][0]['rings'] consists of 768 points.


**Step 2:** We perform the distance computations using Pyproj

One issue in performing geodetic computation is that any (all) geographic coordinate systems are eventually translated to the surface of the earth - which is not flat. That means every computation of distance between two points is some kind of arc (not actually a straight line). Further the earth is not a true sphere, its a type of ellipsoid. That means the amount of curvature varies depending upon where you are on the surface and the direction - which changes the distance.

Lucky for us there are geographers who like to write code and have built systems to simplify the computation of distances over the earth's surface. One of those systems is called [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html). It has functions that will convert coordinate points between (almost) any two of the many different geographic coordinate systems. As well, Pyproj provides ways to compute distances between two points (mostly assuming the points are already in the same coordinate system).

Here we use the Geod() object to calculate the distance between our slected starting city and the location of the Wildfire.


In [ ]:

#
#    A dictionary of my assigned city
CITY_LOCATIONS = {
    'Mesa' :     {'city'   : 'Mesa',
                       'latlon' : [33.40, -111.72] }
}

Note: Convert points between geodetic coordinate systems

One of the constraints in doing geodetic computations is that most of the time we need to have our points (the coordinates for places) in the same geographic coordinate system. There are tons and tons of coordinate systems. You can find descriptions of many of them at [EPSG.io](https://epsg.io).

In the code below, we take the geometry of a fire feature, extract the largest ring (i.e., the largest boundary of the fire) and convert all of the points in that ring from the ESRI:102008 coordinate system to EPSG:4326 coordinates.

In [ ]:
# Taken fro the professors code
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

In [ ]:
# # Taken fro the professors code
#
#   Convert one ring from the default to EPSG
#
#   There are two options here - depending upon whether you loaded data useing GeoJSON or the wildfire.Reader
#
ring_in_epsg4326 = convert_ring_to_epsg4326(gj_bigest_ring)
#
#ring_in_epsg4326 = convert_ring_to_epsg4326(wf_bigest_ring)
#
print(f"Ring consists of {len(ring_in_epsg4326)} points.")
#
#    If you want to print them out you can see what they look like converted.
#print(ring_in_epsg4326)
#for point in ring_in_epsg4326:
#    print(f"{point[0]},{point[1]}")


Ring consists of 768 points.


## Compute distance between Mesa and wildfires

The basic problem is knowing how far away a fire is from some location (like a city). One issue is that fires are irregularly shaped so the actual answer to that is a bit dependent upon the exact shape and how you want to think about the notion of 'distance'. For example, should we just find the closest point on the perimiter of a fire and call that the distance? Maybe we should find the centroid of the region, identify that as a geolocation (coordinate) and then calculate the distance to that? We can come up with numerous other ways.

The first bit of code finds the point on the perimiter with the shortest distance to the city (place) and returns the distance as well as the lat,lon of the perimeter point.

The second bit of code calculates the average distance of all perimeter points to the city (place) and returns that average as the distance. This is not quite what the centroid would be, but it is probably fairly close.

I have initally calculated both the distances to understand if the number of fires with our required range of 650 changes, and then decided to use the shortest distance for my further analysis given that the number of fires is higher that way.


In [ ]:
# Taken fro the professors code
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point


# Taken fro the professors code
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average



In [ ]:
# Attribution: The code below was adapted from the professors code and from Navya Eedula's code.

wildfire_data = list()

# I decided to select the following columns since I wanted the 
# liberity to decide which ones are relevant to my analysis and also keeping the other parts of the project in mind.
selected_columns = ['OBJECTID', 'USGS_Assigned_ID', 'Assigned_Fire_Type', 'Fire_Year', 'Fire_Polygon_Tier', 
    'Fire_Attribute_Tiers', 'GIS_Acres', 'GIS_Hectares', 'Listed_Fire_Types', 'Listed_Fire_Names', 'Listed_Fire_Codes', 
    'Listed_Fire_IDs', 'Listed_Fire_Dates', 'Listed_Fire_Causes', 'Listed_Fire_Cause_Class', 'Listed_Rx_Reported_Acres', 
    'Circleness_Scale', 'Circle_Flag', 'Shape_Length', 'Shape_Area']

Mesa = CITY_LOCATIONS["Mesa"]

for i, fire in enumerate(filtered_wildfires):

    fire_info = fire['attributes']
    
    # Get geometry
    if 'rings' in fire['geometry']:
        ring_data = fire['geometry']['rings'][0]
    elif 'curveRings' in fire['geometry']:
        ring_data = fire['geometry']['curveRings'][0]
    else:
        raise Exception("HEY! No compatible geometry in this fire data!!!")

    try:
        shortest = shortest_distance_from_place_to_fire_perimeter(Mesa['latlon'], ring_data)

        wildfire_record = {key: fire_info[key] for key in selected_columns}
        wildfire_record["Min_Distance"] = shortest[0]

        avg_dist = average_distance_from_place_to_fire_perimeter(Mesa['latlon'], ring_data)
        wildfire_record["Average_Distance"] = avg_dist

        wildfire_data.append(wildfire_record)
    except Exception as ex:
        print(f"Error processing wildfire at index {i}, ID: {fire_info['USGS_Assigned_ID']}")
        print(ex)

Error processing wildfire at index 91887, ID: 109605
0
Error processing wildfire at index 92506, ID: 110224
0
Error processing wildfire at index 92921, ID: 110639
0
Error processing wildfire at index 93713, ID: 111431
0
Error processing wildfire at index 94179, ID: 111897
0
Error processing wildfire at index 94692, ID: 112410
0
Error processing wildfire at index 94697, ID: 112415
0
Error processing wildfire at index 95693, ID: 113411
0
Error processing wildfire at index 95947, ID: 113665
0
Error processing wildfire at index 96020, ID: 113738
0
Error processing wildfire at index 96048, ID: 113766
0
Error processing wildfire at index 96087, ID: 113805
0
Error processing wildfire at index 96591, ID: 114309
0
Error processing wildfire at index 96604, ID: 114322
0
Error processing wildfire at index 97911, ID: 115629
0
Error processing wildfire at index 98256, ID: 115974
0
Error processing wildfire at index 98517, ID: 116235
0
Error processing wildfire at index 99368, ID: 117086
0
Error proc

We can see from the output above that about 28 fires could not be processed. This is likely becuase of irregular geometry. I decided to proceed with these given that the dataset is pretty huge and this is not even 1% of its size

In [ ]:
complete_wildfires_df = pd.DataFrame(wildfire_data)

#  Here I am finding the minimum and maximum distance in the Min_Distance column from the whole dataset
min_distance = complete_wildfires_df['Min_Distance'].min()
max_distance = complete_wildfires_df['Min_Distance'].max()

print(f"Minimum Distance: {min_distance} miles")
print(f"Maximum Distance: {max_distance} miles")

# Filter based on distance less than or equal to 650 miles
filtered_df_min_dist = complete_wildfires_df[complete_wildfires_df['Min_Distance'] <= 650]
filtered_df_avg_dist =  complete_wildfires_df[complete_wildfires_df['Average_Distance'] <= 650]

print("Filtered DataFrame where Min_Distance <= 650 miles):", filtered_df_min_dist.shape)
print("Filtered DataFrame where Average_Distance <= 650 miles):", filtered_df_avg_dist.shape)

Minimum Distance: 8.157650682154788 miles
Maximum Distance: 3511.43060816707 miles
Filtered DataFrame where Min_Distance <= 650 miles): (33901, 22)
Filtered DataFrame where Average_Distance <= 650 miles): (33848, 22)


Based on the above numbers, I found that filtered_df_min_dist has more records than filtered_df_avg_dist. It makes sense since it is the minimum distance to the fire perimeter and not like the centeroid. 

In [ ]:
# Save the filtered and complete wildfires DataFrame
complete_wildfires_df.to_csv("./Processed Data/complete_wildfires_with_distances.csv")
filtered_df_min_dist.to_csv("./Processed Data/filtered_wildfires_with_distances.csv")

I save the complete dataframe to ensure I have the data for other analysis and visualization